In [1]:
#imports and set up

from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from IPython.display import display
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import time
import sys
from process_pickle import process_pickle
%matplotlib inline
from ipywidgets import interact, interactive, fixed #Sliders for image selection
import ipywidgets as widgets
mpl.rc('figure',  figsize=(4.77, 2.95))
mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rcParams['lines.linewidth'] = 0.6
mpl.rcParams['lines.color'] = 'r'
plt.rc('grid', linestyle="--", color='gray')

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  

import pims
import trackpy as tp
import os

In [3]:
# function that is called to configure how plots are to be drawn

def setup_plot():
    ax = plt.gca()
    SPINE_COLOR = 'gray'
    for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)
    plt.grid()
    fig = plt.gcf()
    fig.set_size_inches(4.77, 2.95, forward=True)

Use the file picker to locate the .pkl containing the particle trajectories

or select a previously output .xlsx to skip the calculations

**If it doesn't appear you probably need to minimise your browser or alt tab around for it**

In [4]:
set_custom_data_directory = True

In [5]:
# select the .pkl files you wish to analyse. When you're done selecting cancel should
# allow you to continue to the next cell.

import tkinter as tk
from tkinter import filedialog

pkl_to_process = []
file_path = "1"
while len(file_path) != 0:
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if len(file_path) != 0:
        pkl_to_process.append(file_path)

In [6]:
# confirm you've selected the correct files

pkl_to_process

['C:/Users/lc16629/OneDrive - University of Bristol/Desktop/Python/DATA/22-1-18 0.8gL 25C (Cropped)/data output/trajectories.pkl']

In [7]:
calculate = True

In [8]:
# configure the calculations to be carried out on the data contained in the .pkl files
# if you don't enter "NEXT" at the end you can queue another set of calculations to be carried out on the 
# same .pkl, e.g with a frame lookback.
pkl_param = []

i = 0
mpp = 15 # microns per pixel
timestep = 30 # seconds between images

while i < len(pkl_to_process):
    print(pkl_to_process[i])
    print("Setting Parameters...")
    # uncomment the below lines to set mpp or timestep on a per .pkl basis
    #mpp = int(input("Microns per Pixel: "))
    #timestep = int(input("Time between frames in seconds: "))
    frame_lookback = int(input("How many frames to look back for velocity calculations? "))
    pkl_param.append([i,mpp,timestep,frame_lookback])
    move_next = input("Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle")
    if move_next == "NEXT":
        i += 1
    

C:/Users/lc16629/OneDrive - University of Bristol/Desktop/Python/DATA/22-1-18 0.8gL 25C (Cropped)/data output/trajectories.pkl
Setting Parameters...


How many frames to look back for velocity calculations?  25
Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle NEXT


In [9]:
pkl_param

[[0, 15, 30, 25]]

In [10]:
# for the sets of calculations you've requested, select folders to store the results for each set in.

data_dirs = []
data_dir = True
counter = 0
for i in pkl_param:
    root = tk.Tk()
    root.withdraw()
    data_dir = filedialog.askdirectory(title="Pick data dir for {} {}".format(pkl_to_process[i[0]], i[3]))
    data_dirs.append(data_dir)

In [26]:
def pick_path():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askdirectory()
    return file_path

# select the directory containing the pictures you want to process
image_directory = pick_path()

# import the images into the "frames" variable - only .tif files are selected
frames = pims.ImageSequence(image_directory + "/*.tif", as_grey=True)

In [11]:
# if the folders you've requested the results stored in don't exist, make them:

for directory in data_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [13]:
for c, param in enumerate(pkl_param):
    # the process_pickle function is contained within a separate file - process_pickle.py
    df = process_pickle(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])

 Calculating Displacement for particles: 7.97%

KeyboardInterrupt: 

In [30]:
from scipy.interpolate import griddata
tstep = 30 # in Seconds - 60 for 1min, 30 for 30s

def fiftypx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/50
    xdf = np.rint(xdf)
    xdf = xdf*50
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/50
    ydf = np.rint(ydf)
    ydf = ydf*50

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    print('Dataset has total of %s frames and %s particles.' % (nofrm, noprt))
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 17)
    ys = np.linspace(0, 1600, 33)
    xi,yi = np.meshgrid(xs, ys)    
        
    v_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    pt_count=np.full((ys.shape[0],xs.shape[0],nofrm),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/50)
            y = int((ydf[f, p])/50)
                            
            v_vals[y,x,f,p] = vdf[f, p]
    
    for x in range(0,17):
        for y in range(0,33):
            for f in range(0, nofrm-1):
                pt_count[y,x,f] = np.count_nonzero(~np.isnan(v_vals[y,x,f,:]))
                
        
    v_vals=np.nanmean(v_vals, axis=3)
                
    maxp = np.nanmax(pt_count)
    
    if not os.path.exists(general_directory + "/heatmap_frames_50px/"):
        os.makedirs(general_directory + "/heatmap_frames_50px/")

    for i in range(0, nofrm-1, npstep): 
    
        fig = plt.figure()
        
#         axf = fig.add_subplot(1, 3, 1)
#         axf.axes.get_xaxis().set_visible(False)
#         axf.axes.get_yaxis().set_visible(False)
#         plt.imshow(frames[i], cmap='gray')
        
        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 10.5)
        axp.set_ylim(-0.5, 26.5)
        plt.imshow(pt_count[:,:,i], cmap='RdBu_r', vmin=0, vmax=maxp)
        for a in range(0,11):
            for b in range(0,27):
                text = axp.text(a, b, int(pt_count[b, a, i]), fontsize=5, ha="center", va="center", color="k")
        
        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 10.5)
        axh.set_ylim(-0.5, 26.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(wspace=1)
    
        im = plt.imshow(v_vals[:,:,i], cmap='RdBu_r', vmin=vmn, vmax=vmx)
        
        fig.suptitle("t = " + (str(round(((i*tstep)/3600), 1))) + " hrs") 
        axh.set_title("Velocity (Normalised, log scale)")
#         axf.set_title("Sample image")
        axp.set_title("Particle count")
        
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 
        
        plt.savefig(general_directory + "/heatmap_frames_50px/heatmap_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_50px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_50px/heatmap.gif"), heatim)
    

def hundrdpx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/100
    xdf = np.rint(xdf)
    xdf = xdf*100
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/100
    ydf = np.rint(ydf)
    ydf = ydf*100

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 9)
    ys = np.linspace(0, 1600, 17)
    xi,yi = np.meshgrid(xs, ys)    
        
    v_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    pt_count=np.full((ys.shape[0],xs.shape[0],nofrm),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/100)
            y = int((ydf[f, p])/100)
                            
            v_vals[y,x,f,p] = vdf[f, p]
    
    for x in range(0,9):
        for y in range(0,17):
            for f in range(0, nofrm-1):
                pt_count[y,x,f] = np.count_nonzero(~np.isnan(v_vals[y,x,f,:]))
                
    maxp = np.nanmax(pt_count)
    
    v_vals=np.nanmean(v_vals, axis=3)
    
    if not os.path.exists(general_directory + "/heatmap_frames_100px/"):
        os.makedirs(general_directory + "/heatmap_frames_100px/")

    for i in range(0, nofrm-1, npstep): 
    
        fig = plt.figure()
        
#         axf = fig.add_subplot(1, 3, 1)
#         axf.axes.get_xaxis().set_visible(False)
#         axf.axes.get_yaxis().set_visible(False)
#         plt.imshow(frames[i], cmap='gray')
        
        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 4.5)
        axp.set_ylim(-0.5, 12.5)
        plt.imshow(pt_count[:,:,i], cmap='RdBu_r', vmin=0, vmax=maxp)
        for a in range(0,5):
            for b in range(0,13):
                text = axp.text(a, b, int(pt_count[b, a, i]), fontsize=8, ha="center", va="center", color="k")
        
        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 4.5)
        axh.set_ylim(-0.5, 12.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(wspace=1)
    
        im = plt.imshow(v_vals[:,:,i], cmap='RdBu_r', vmin=vmn, vmax=vmx)
        
        fig.suptitle("t = " + (str(round(((i*tstep)/3600), 1))) + " hrs")
        axh.set_title("Velocity (Normalised, log10)")
#         axf.set_title("Sample image")
        axp.set_title("Particle count")
        
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical')        
        
        plt.savefig(general_directory + "/heatmap_frames_100px/heatmap_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_100px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_100px/heatmap.gif"), heatim)    
    
def twohundrdpx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/200
    xdf = np.rint(xdf)
    xdf = xdf*200
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/200
    ydf = np.rint(ydf)
    ydf = ydf*200

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    print('Dataset has total of %s frames and %s particles.' % (nofrm, noprt))
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 5)
    ys = np.linspace(0, 1600, 9)
    xi,yi = np.meshgrid(xs, ys)    
        
    v_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    pt_count=np.full((ys.shape[0],xs.shape[0],nofrm),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/200)
            y = int((ydf[f, p])/200)
                            
            v_vals[y,x,f,p] = vdf[f, p]
            
            
    for x in range(0,5):
        for y in range(0,9):
            for f in range(0, nofrm-1):
                pt_count[y,x,f] = np.count_nonzero(~np.isnan(v_vals[y,x,f,:]))
    
    v_vals=np.nanmean(v_vals, axis=3)
    
    maxp = np.nanmax(pt_count)
    
    if not os.path.exists(general_directory + "/heatmap_frames_200px/"):
        os.makedirs(general_directory + "/heatmap_frames_200px/")

    for i in range(0, nofrm-1, npstep): 
    
        fig = plt.figure()
        
#         axf = fig.add_subplot(1, 3, 1)
#         axf.axes.get_xaxis().set_visible(False)
#         axf.axes.get_yaxis().set_visible(False)
#         plt.imshow(frames[i], cmap='gray')
        
        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 1.5)
        axp.set_ylim(-0.5, 5.5)
        plt.imshow(pt_count[:,:,i], cmap='RdBu_r', vmin=0, vmax=maxp)
        for a in range(0,2):
            for b in range(0,6):
                text = axp.text(a, b, int(pt_count[b, a, i]), fontsize=8, ha="center", va="center", color="k")
        
        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 1.5)
        axh.set_ylim(-0.5, 5.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(wspace=1)
    
        im = plt.imshow(v_vals[:,:,i], cmap='RdBu_r', vmin=vmn, vmax=vmx)
        
        fig.suptitle("t = " + (str(round(((i*tstep)/3600), 1))) + " hrs")
        axh.set_title("Velocity (Normalised, log10)")
#         axf.set_title("Sample image")
        axp.set_title("Particle count")
        
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical')  
        
        plt.savefig(general_directory + "/heatmap_frames_200px/heatmap_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_200px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_200px/heatmap.gif"), heatim)     
    

for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    fiftypx_plots(data_dirs[c])
    print('\n')
    hundrdpx_plots(data_dirs[c])
    print('\n')
    twohundrdpx_plots(data_dirs[c])
    print('Done!')
 



Processing data: 


Dataset has total of 8479 frames and 3265 particles.


MemoryError: 

In [17]:
from scipy.interpolate import griddata
tstep = 30 # in Seconds - 60 for 1min, 30 for 30s

def fiftypx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/50
    xdf = np.rint(xdf)
    xdf = xdf*50
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/50
    ydf = np.rint(ydf)
    ydf = ydf*50

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    print('Dataset has total of %s frames and %s particles.' % (nofrm, noprt))
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 17)
    ys = np.linspace(0, 1600, 33)
    xi,yi = np.meshgrid(xs, ys)    
        
    for f in range(0, nofrm-1, npstep):   
        
        frame = int(f)
        
        print('frame = %s' % frame)
        
        v_vals=np.full((ys.shape[0],xs.shape[0],noprt),np.nan)
        pt_count=np.full((ys.shape[0],xs.shape[0]),np.nan)

        for p in range(1, noprt-1):
                x = int((xdf[f, p])/50)
                y = int((ydf[f, p])/50)

                v_vals[y,x,p] = vdf[f, p]

        for x in range(0,17):
            for y in range(0,33):
                for f in range(0, nofrm-1):
                    pt_count[y,x] = np.count_nonzero(~np.isnan(v_vals[y,x,:]))


        v_vals=np.nanmean(v_vals, axis=2)

        if not os.path.exists(general_directory + "/heatmap_frames_50px/"):
            os.makedirs(general_directory + "/heatmap_frames_50px/") 

        fig = plt.figure()

        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 10.5)
        axp.set_ylim(26.5, -0.5)
        plt.imshow(pt_count[:,:], cmap='RdBu_r', vmin=0, vmax=10)
        for a in range(0,11):
            for b in range(0,27):
                text = axp.text(a, b, int(pt_count[b, a]), fontsize=5, ha="center", va="center", color="k")

        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 10.5)
        axh.set_ylim(26.5, -0.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)

        plt.subplots_adjust(wspace=1)

        im = plt.imshow(v_vals[:,:], cmap='RdBu_r', vmin=vmn, vmax=vmx)

        fig.suptitle("t = " + (str(round(((frame*tstep)/3600), 1))) + " hrs") 
        axh.set_title("Velocity (Normalised, log scale)")
        axp.set_title("Particle count")

        im.set_clim(vmax=vmx, vmin=vmn)

        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 

        plt.savefig(general_directory + "/heatmap_frames_50px/heatmap_" + str(frame) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_50px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_50px/heatmap.gif"), heatim)
    

def hundrdpx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/100
    xdf = np.rint(xdf)
    xdf = xdf*100
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/100
    ydf = np.rint(ydf)
    ydf = ydf*100

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 9)
    ys = np.linspace(0, 1600, 17)
    xi,yi = np.meshgrid(xs, ys)    
    
    for f in range(0, nofrm-1, npstep):     
        
        frame = int(f)
        
        print('frame = %s' % frame)
        
        v_vals=np.full((ys.shape[0],xs.shape[0],noprt),np.nan)
        pt_count=np.full((ys.shape[0],xs.shape[0]),np.nan)

        for p in range(1, noprt-1):
                x = int((xdf[f, p])/100)
                y = int((ydf[f, p])/100)

                v_vals[y,x,p] = vdf[f, p]

        for x in range(0,9):
            for y in range(0,17):
                for f in range(0, nofrm-1):
                    pt_count[y,x] = np.count_nonzero(~np.isnan(v_vals[y,x,:]))


        v_vals=np.nanmean(v_vals, axis=2)

        if not os.path.exists(general_directory + "/heatmap_frames_100px/"):
            os.makedirs(general_directory + "/heatmap_frames_100px/") 

        fig = plt.figure()

        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 4.5)
        axp.set_ylim(12.5, -0.5)
        plt.imshow(pt_count[:,:], cmap='RdBu_r', vmin=0, vmax=20)
        for a in range(0,5):
            for b in range(0,13):
                text = axp.text(a, b, int(pt_count[b, a]), fontsize=5, ha="center", va="center", color="k")

        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 4.5)
        axh.set_ylim(12.5, -0.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)

        plt.subplots_adjust(wspace=1)

        im = plt.imshow(v_vals[:,:], cmap='RdBu_r', vmin=vmn, vmax=vmx)

        fig.suptitle("t = " + (str(round(((frame*tstep)/3600), 1))) + " hrs") 
        axh.set_title("Velocity (Normalised, log scale)")
        axp.set_title("Particle count")

        im.set_clim(vmax=vmx, vmin=vmn)

        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 

        plt.savefig(general_directory + "/heatmap_frames_100px/heatmap_" + str(frame) + ".png")
        plt.close()
               
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_100px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_100px/heatmap.gif"), heatim)    
    
def twohundrdpx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/200
    xdf = np.rint(xdf)
    xdf = xdf*200
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/200
    ydf = np.rint(ydf)
    ydf = ydf*200

    vdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 5)
    ys = np.linspace(0, 1600, 9)
    xi,yi = np.meshgrid(xs, ys) 
    
    for f in range(0, nofrm-1, npstep):
        
        frame = int(f)
        
        print('frame = %s' % frame)
        
        v_vals=np.full((ys.shape[0],xs.shape[0],noprt),np.nan)
        pt_count=np.full((ys.shape[0],xs.shape[0]),np.nan)

        for p in range(1, noprt-1):
                x = int((xdf[f, p])/200)
                y = int((ydf[f, p])/200)

                v_vals[y,x,p] = vdf[f, p]

        for x in range(0,17):
            for y in range(0,33):
                for f in range(0, nofrm-1):
                    pt_count[y,x] = np.count_nonzero(~np.isnan(v_vals[y,x,:]))


        v_vals=np.nanmean(v_vals, axis=2)

        if not os.path.exists(general_directory + "/heatmap_frames_200px/"):
            os.makedirs(general_directory + "/heatmap_frames_200px/") 

        fig = plt.figure()

        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 1.5)
        axp.set_ylim(5.5, -0.5)
        plt.imshow(pt_count[:,:], cmap='RdBu_r', vmin=0, vmax=40)
        for a in range(0,2):
            for b in range(0,6):
                text = axp.text(a, b, int(pt_count[b, a]), fontsize=5, ha="center", va="center", color="k")

        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 1.5)
        axh.set_ylim(5.5, -0.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)

        plt.subplots_adjust(wspace=1)

        im = plt.imshow(v_vals[:,:], cmap='RdBu_r', vmin=vmn, vmax=vmx)

        fig.suptitle("t = " + (str(round(((frame*tstep)/3600), 1))) + " hrs") 
        axh.set_title("Velocity (Normalised, log scale)")
        axp.set_title("Particle count")

        im.set_clim(vmax=vmx, vmin=vmn)

        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 

        plt.savefig(general_directory + "/heatmap_frames_200px/heatmap_" + str(frame) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames_200px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames_200px/heatmap.gif"), heatim)     
    

for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    fiftypx_plots(data_dirs[c])
    print('\n')
    hundrdpx_plots(data_dirs[c])
    print('\n')
    twohundrdpx_plots(data_dirs[c])
    print('Done!')
 



Processing data: 


Dataset has total of 8479 frames and 3265 particles.
frame = 0


C:\Users\lc16629\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


frame = 25
frame = 50
frame = 75
frame = 100
frame = 125
frame = 150
frame = 175
frame = 200
frame = 225
frame = 250
frame = 275
frame = 300
frame = 325
frame = 350
frame = 375
frame = 400
frame = 425
frame = 450
frame = 475
frame = 500
frame = 525
frame = 550
frame = 575
frame = 600
frame = 625
frame = 650
frame = 675
frame = 700
frame = 725
frame = 750
frame = 775
frame = 800
frame = 825
frame = 850
frame = 875
frame = 900
frame = 925
frame = 950
frame = 975
frame = 1000
frame = 1025
frame = 1050
frame = 1075
frame = 1100
frame = 1125
frame = 1150
frame = 1175
frame = 1200
frame = 1225
frame = 1250
frame = 1275
frame = 1300
frame = 1325
frame = 1350
frame = 1375
frame = 1400
frame = 1425
frame = 1450
frame = 1475
frame = 1500
frame = 1525
frame = 1550
frame = 1575
frame = 1600
frame = 1625
frame = 1650
frame = 1675
frame = 1700
frame = 1725
frame = 1750
frame = 1775
frame = 1800
frame = 1825
frame = 1850
frame = 1875
frame = 1900
frame = 1925
frame = 1950
frame = 1975
frame = 2000
fr

C:\Users\lc16629\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:179: RuntimeWarning: Mean of empty slice


frame = 25
frame = 50
frame = 75
frame = 100
frame = 125
frame = 150
frame = 175
frame = 200
frame = 225
frame = 250
frame = 275
frame = 300
frame = 325
frame = 350
frame = 375
frame = 400
frame = 425
frame = 450
frame = 475
frame = 500
frame = 525
frame = 550
frame = 575
frame = 600
frame = 625
frame = 650
frame = 675
frame = 700
frame = 725
frame = 750
frame = 775
frame = 800
frame = 825
frame = 850
frame = 875
frame = 900
frame = 925
frame = 950
frame = 975
frame = 1000
frame = 1025
frame = 1050
frame = 1075
frame = 1100
frame = 1125
frame = 1150
frame = 1175
frame = 1200
frame = 1225
frame = 1250
frame = 1275
frame = 1300
frame = 1325
frame = 1350
frame = 1375
frame = 1400
frame = 1425
frame = 1450
frame = 1475
frame = 1500
frame = 1525
frame = 1550
frame = 1575
frame = 1600
frame = 1625
frame = 1650
frame = 1675
frame = 1700
frame = 1725
frame = 1750
frame = 1775
frame = 1800
frame = 1825
frame = 1850
frame = 1875
frame = 1900
frame = 1925
frame = 1950
frame = 1975
frame = 2000
fr

IndexError: index 9 is out of bounds for axis 0 with size 9

In [ ]:
from scipy.interpolate import griddata

def fiftypx_plots(general_directory):
    npstep = frame_lookback
    xdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/50
    xdf = np.rint(xdf)
    xdf = xdf*50
    
    ydf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/50
    ydf = np.rint(ydf)
    ydf = ydf*50

    vdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='hyp_vel').values
    nofrm = len(vdf)
    noprt = len(vdf[0])
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    vdf[np.isnan(vdf)] = 0
    vdf[vdf>=1e-4] = np.nan
    vdf[vdf==0] = np.nan
    
    vdf = np.log10(vdf)
    vdf = (vdf + abs(np.nanmin(vdf)))
    vdf = vdf/(np.nanmax(vdf))
    vmx = np.nanmax(vdf)
    vmn = np.nanmin(vdf)
                
    xs = np.linspace(0, 800, 17)
    ys = np.linspace(0, 1600, 33)
    xi,yi = np.meshgrid(xs, ys)    
        
    v_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    pt_count=np.full((ys.shape[0],xs.shape[0],nofrm),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/50)
            y = int((ydf[f, p])/50)
                            
            v_vals[y,x,f,p] = vdf[f, p]
    
    for x in range(0,17):
        for y in range(0,33):
            for f in range(0, nofrm-1):
                pt_count[y,x,f] = np.count_nonzero(~np.isnan(v_vals[y,x,f,:]))
                
        
    v_vals=np.nanmean(v_vals, axis=3)
                
    maxp = np.nanmax(pt_count)
    
    if not os.path.exists(general_directory + "/no_sample_pic/heatmap_frames_50px/"):
        os.makedirs(general_directory + "/no_sample_pic/heatmap_frames_50px/")

    for i in range(0, nofrm-1, npstep): 
    
        fig = plt.figure()
        
        axp = fig.add_subplot(1, 2, 2)
        axp.axes.get_xaxis().set_visible(False)
        axp.axes.get_yaxis().set_visible(False)
        axp.patch.set_facecolor('black')
        axp.set_xlim(-0.5, 10.5)
        axp.set_ylim(-0.5, 26.5)
        plt.imshow(pt_count[:,:,i], cmap='RdBu_r', vmin=0, vmax=maxp)
        for a in range(0,11):
            for b in range(0,27):
                text = axp.text(a, b, int(pt_count[b, a, i]), fontsize=5, ha="center", va="center", color="k")
        
        axh = fig.add_subplot(1, 2, 1)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 10.5)
        axh.set_ylim(-0.5, 26.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(wspace=1)
    
        im = plt.imshow(v_vals[:,:,i], cmap='RdBu_r', vmin=vmn, vmax=vmx)

        axh.set_title("Velocity (Normalised, log10)")
        axp.set_title("Particle count")
        
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 
        
        plt.savefig(general_directory + "/no_sample_pic/heatmap_frames_50px/heatmap_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/no_sample_pic/heatmap_frames_50px/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/no_sample_pic/heatmap_frames_50px/heatmap.gif"), heatim)    

for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    fiftypx_plots(data_dirs[c])
    print('Done!')
 



Processing data: 




In [50]:
import matplotlib.mlab as mlab
def msd_map(general_directory):
    npstep = 10
    xdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    x_exact = np.copy(xdf)
    xdf = xdf/50
    xdf = np.rint(xdf)
    xdf = 50*xdf
    
    ydf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    y_exact = np.copy(ydf)
    ydf = ydf/50
    ydf = np.rint(ydf)
    ydf = 50*ydf
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
   
    
    
    vxdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 16])

    vxdf = vxdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x_velocity').values

    vydf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 15])
    vydf = vydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y_velocity').values
    
    nofrm = len(vxdf)
    noprt = len(vxdf[0])
    msd = np.empty((nofrm,noprt,))
    msd[:] = np.nan
    
    for p in range(1, noprt-1): # Chi-squared and heterogeneity type test for each particle in each frame
        for f in range(1, nofrm-1):
            x0 = x_exact[(f-1),p]
            xt = x_exact[f,p]
            y0 = y_exact[(f-1),p]
            yt = y_exact[f,p]
            
            x2 = (xt-x0)**2 # Chi squared test for Vx
            y2 = (yt-y0)**2 # Chi squared test for Vy
            msd[f,p] = (x2+y2)/2 # I2 heterogeneity value, for 2 degrees of freedom
            
    msd = np.log10(msd)
    msd = (msd + abs(np.nanmin(msd)))
    msd = msd/(np.nanmax(msd))
    vmx = np.nanmax(msd)
    vmn = np.nanmin(msd)
    
    xs = np.linspace(0, 800, 17)
    ys = np.linspace(0, 1600, 33)
    xi,yi = np.meshgrid(xs, ys)    
        
    msd_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/50)
            y = int((ydf[f, p])/50)
                            
            msd_vals[y,x,f,p] = msd[f, p]
    
    msd_vals=np.nanmean(msd_vals, axis=3)
    
    if not os.path.exists(general_directory + "/msd_map/"):
        os.makedirs(general_directory + "/msd_map/")

    for i in range(1, nofrm-1, npstep): 
        
        plt.close("all")
    
        fig = plt.figure()
        
        axf = fig.add_subplot(1, 2, 1)
        axf.set_xlim(0, 1)
        axf.set_ylim(0, 10)
        plt.hist(msd_vals[:,:,i].flatten(), bins=20)
        
        axh = fig.add_subplot(1, 2, 2)
        axh.patch.set_facecolor('black')
        axh.set_xlim(-0.5, 10.5)
        axh.set_ylim(26.5, -0.5)  
        axh.axes.get_xaxis().set_visible(False)
        axh.axes.get_yaxis().set_visible(False)
        
        plt.subplots_adjust(wspace=0.3)
    
        im = plt.imshow(msd_vals[:,:,i], cmap='RdBu_r', vmin=vmn, vmax=vmx)
        
        fig.suptitle("t = " + (str(i*30)) + "s") 
        axh.set_title("<r\u00b2> (Normalised, log10)")
        axf.set_title("MSD Distribution")
        
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical') 
        
        plt.savefig(general_directory + "/msd_map/img_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(1, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/msd_map/img_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/msd_map/anim.gif"), heatim)
    
for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    msd_map(data_dirs[c])
    print('\n')
    print('Done!')



Processing data: 




C:\Users\lc16629\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: Mean of empty slice




Done!
